<p>&nbsp; <br><b>Part of the battle of writing Who is Active is achieving the proper level of balance between enough information and too much information</b>. </p>
  
<p>It’s important to return sufficient data to help debug the most common problems without users having to tweak the parameters. And it’s important to restrict the amount of data sent back so that the default output is not overwhelming, nor is the performance of the procedure sacrificed.</p>
  
<p>Following are the current default columns, broken into four basic categories:</p>
  
<blockquote>   
<p><b><u>Time and Status</u></b></p>
    
<p><font face="Courier New">[dd hh:mm:ss.mss]       <br>[start_time]        <br>[percent_complete]        <br>[collection_time]        <br>[status]        <br></font>&nbsp;</p>
    
<p><b><u>Identifiers</u></b></p>
    
<p><font face="Courier New">[session_id]       <br>[request_id]        <br>[login_name]        <br>[host_name]        <br>[database_name]        <br>[program_name]        <br></font>&nbsp;</p>
    
<p><b><u>Things Slowing Down Your Query</u></b></p>
    
<p><font face="Courier New">[wait_info]       <br>[blocking_session_id]        <br></font>&nbsp;</p>
    
<p><b><u>Things Your Session is Doing</u></b></p>
    
<p><font face="Courier New">[sql_text]       <br>[CPU]        <br>[tempdb_allocations]        <br>[tempdb_current]        <br>[reads]        <br>[writes]        <br>[physical_reads]        <br>[used_memory]        <br>[open_tran_count]</font></p>
 </blockquote>
  
<p>&nbsp;</p>
  
<p>Each set of columns deserves some description, and we’ll start with <b>Time and Status</b>. These columns tell you how long your query has been running ([start_time] and its cousin, the “convenience column” [dd hh:mm:ss.mss]), how much longer things might be running ([percent_complete]), whether anything is running at all ([status]), and a record of when you asked ([collection_time]). </p>
  
<p>The <b>Identifiers</b> are ways of telling one session—or class of sessions—apart from another. The [session_id] and [request_id] columns are, of course, SQL Server’s way of doing this, while the rest of the columns are more human-readable. Note that [request_id] will almost always have a value of 0 for active requests (those where the [status] column has any value other than “sleeping”), and NULL for sleeping sessions. This is not quite the same as the way the data is represented in the sysprocesses DMV, but I don’t think it makes sense to have any [request_id] when there is no request. To see a value greater than 0, you’ll have to use MARS in your application. Not a common thing, which is why this column shows up on the far righthand side of the output.</p>
  
<p>The <b>Things Slowing Down Your Query</b> columns describe wait states and information about blocking. I’ll get into these in detail in a later post.</p>
  
<p>Finally, the <b>Things Your Session is Doing</b> columns give information about what is happening, or has happened, on behalf of your session. At this point in the series it’s worth sharing further information about a few of the less obvious of these columns:</p>
  
<ul>   
<li><b>The most confusing of these columns are those related to tempdb</b>. Each of the columns reports a number of 8 KB pages. The [tempdb_allocations] column is collected directly from the tempdb-related DMVs, and indicates how many pages were allocated in tempdb due to temporary tables, LOB types, spools, or other consumers. The [tempdb_current] column is computed by subtracting the deallocated pages information reported by the tempdb DMVs from the number of allocations. Seeing a high number of allocations with a small amount of current pages means that your query may be slamming tempdb, but is not causing it to grow. Seeing a large number of current pages means that your query may be responsible for all of those auto-grows you keep noticing.</li>
    
<li>The [used_memory] column is also reported based on a number of 8 KB pages. The number a combination of both procedure cache memory and workspace memory grant. </li>
    
<li><b>[open_tran_count] is by far the most useful column that Who is Active pulls from the deprecated sysprocesses view</b>. And only from that view, since Microsoft has not bothered replicating it elsewhere. It can be used not only to tell whether the session has an active transaction, but also to find out how deeply nested the transaction is. This is invaluable information when debugging situations where applications open several nested transactions and don’t send enough commits to seal the deal.</li>